In [2]:
# file load
#%matplotlib inline
%matplotlib
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

train = pd.read_csv("raw/train.csv") # 136*43 matrix
test = pd.read_csv("raw/test.csv") # 99999*42 matrix

Using matplotlib backend: MacOSX


In [2]:
# convert date to days
import time
from datetime import datetime as dt
# train
all_diff = []
for item in train["Open Date"]:
    diff = dt.now() - dt.strptime(item, "%m/%d/%Y")
    all_diff.append(int(diff.days/365))

train['DaysOfRest'] = pd.Series(all_diff)

# test
all_diff = []
for item in test["Open Date"]:
    diff = dt.now() - dt.strptime(item, "%m/%d/%Y")
    all_diff.append(int(diff.days/365))

test['DaysOfRest'] = pd.Series(all_diff)

In [3]:
# drop id and open date
useless_features = ['Id','Open Date','City Group']
train = train.drop(useless_features, 1)
test = test.drop(useless_features, 1)
print train.keys()
print test.keys()

Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest'], dtype='object')
Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest'], dtype='object')


In [4]:
# add new field = max value
train_normal = train.copy()

names = [name for name in train.keys() if name[0]=='P']
max_params = []
j = 0
for i in range(0,len(train_normal)):
    max_value = 0
    for name in names:
        if train_normal.iloc[i][name] > max_value:
            max_value = train_normal.iloc[i][name]
    if max_value > 5:
        j +=1
    #print max_value
    max_params.append(max_value)
    
train_normal['max_params'] = pd.Series(max_params)
print train_normal.keys()
print train_normal['max_params'].value_counts()
print train_normal.shape

Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest', u'max_params'], dtype='object')
5     114
25     12
15      5
10      2
20      2
18      2
dtype: int64
(137, 42)


In [5]:
# convert strange data to good
names = [name for name in train_normal.keys() if name[0]=='P']
multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,'P11':2,'P12':2,'P13':1.5,'P14':3,
              'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5,
              'P28':2.5,'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}

for i in range(0,len(train_normal)):
    if train_normal.iloc[i]['max_params'] > 5:
        for name in names:
            #train[item] = train[item].astype('category')
            train_normal[name]=train_normal[name].astype(np.float64)
            train_normal.loc[i,name] /= multiplier[name]
    

In [6]:
# normalize test arguments
test_normal = test.copy()
for i in range(0,len(test)):
    for name in names:
        val = test_normal.iloc[i][name]
        if val % 1 != 0 or val > 5:
            test_normal.loc[i,name] /= multiplier[name]
        #if name == 'P7' and val == 2: # probably useless
        #    test_normal.loc[i,name] /= 2
        #if name == 'P29' and val == 5:
        #    test_normal.loc[i,name] /= 2.5
    if i % 5000 == 0:
        print i
print "End"

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
End


In [17]:
print train_normal.loc[:,['City','P1']]

         City  P1
0    İstanbul   4
1      Ankara   4
2       Other   2
3       Other   2
4       Other   3
5      Ankara   2
6    İstanbul   2
7    İstanbul   4
8       Other   1
9       Other   2
10      Other   3
11   İstanbul   2
12     Ankara   2
13   İstanbul   4
14      Other   2
15   İstanbul   4
16   İstanbul   3
17   İstanbul   2
18      İzmir   4
19      Other   2
20      Other   3
21   İstanbul   5
22      Other   3
23      Other   2
24   İstanbul   5
25      Other   4
26     Ankara   3
27      Other   4
28   İstanbul   4
29     Ankara   1
..        ...  ..
107     Other   4
108     Other   1
109    Ankara   2
110     Other   2
111     Other   2
112  İstanbul   4
113     Other   3
114     Other   4
115  İstanbul   2
116     İzmir   2
117    Ankara   3
118  İstanbul   4
119  İstanbul   4
120     Other   2
121     İzmir   2
122  İstanbul   4
123     Other   1
124    Ankara   1
125     Other   3
126     Other   3
127     Other   3
128    Ankara   2
129     Other   4
130     Ot

In [389]:
"""names = [name for name in train.keys() if name[0]=='P']
for i in range(0,len(train)):
    lowest_sum = 1000
    lowest_j = 0
    for j in range(0, len(train)):
        cum_sum = 0
        for name in names:
            cum_sum += np.abs(train.iloc[i][name]- train.iloc[j][name])
        #print outlier_index[i], j,  cum_sum
        if j != i:
            if cum_sum <=lowest_sum:
                lowest_sum = cum_sum
                lowest_j = j
    print "**************"
    print "Nearest item"
    print i, lowest_sum, lowest_j, np.abs(train.iloc[i]["revenue"] - train.iloc[lowest_j]["revenue"])"""

**************
Nearest item
0 23.0 104 1186025.0
**************
Nearest item
1 4.0 91 4839684.0
**************
Nearest item
2 4.0 95 784880.0
**************
Nearest item
3 16.0 110 278575.0
**************
Nearest item
4 23.0 68 479994.0
**************
Nearest item
5 0.0 109 798056.0
**************
Nearest item
6 15.0 115 283650.0
**************
Nearest item
7 7.0 56 2101073.0
**************
Nearest item
8 27.0 115 69512.0
**************
Nearest item
9 8.0 69 1991845.0
**************
Nearest item
10 8.0 129 194669.0
**************
Nearest item
11 17.0 115 278385.0
**************
Nearest item
12 5.0 76 2855789.0
**************
Nearest item
13 3.0 59 1207279.0
**************
Nearest item
14 4.0 76 2591326.0
**************
Nearest item
15 23.0 71 917507.0
**************
Nearest item
16 0.0 38 14235239.0
**************
Nearest item
17 7.0 95 6943025.0
**************
Nearest item
18 18.0 79 975695.0
**************
Nearest item
19 6.0 121 1423142.0
**************
Nearest item
20 27.0 107 2277

In [518]:
# construct new test set
"""new_test = test.copy()
names = new_test.keys()[2:]
numeric_names = [name for name in names if name[0]=='P']
multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,
              'P11':2,'P12':2,'P13':1.5,'P14':3,'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,
              'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5, 'P28':2.5,
              'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}        
print names
i = 1
for rest in one_rest:
    
    sli =  test[test["DaysOfRest"]== rest]
    # set to max
    for name in names:
        val_dict = dict(sli[name].value_counts())
        max_key = max(val_dict,key=val_dict.get) 
        if max_key == 0:
            del(val_dict[max_key])
            max_key = max(val_dict,key=val_dict.get)
        sli.loc[:,name] = max_key
    max_value = np.max(sli.iloc[0][numeric_names])
    
    # convert to normal values
    if max_value > 5:
        print "Big value: {value} {revenue}".format(value = max_value, revenue = rest)
        for name in numeric_names:
            sli.loc[:,name] = sli.iloc[0][name]/multiplier[name]
        
    #new_test.update(sli)
    print "Ready {index}/{total}".format(index = i, total = len(one_rest))
    i+=1
    
for rest in one_rest[0:5]:
    sli =  new_test[new_test["DaysOfRest"]== rest]
    for name in names:
        print name
        val_dict = dict(sli[name].value_counts())
        print val_dict"""

Index([u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest'], dtype='object')
Ready 1/299
Ready 2/299
Ready 3/299
Big value: 25 2567
Ready 4/299
Ready 5/299
Ready 6/299
Ready 7/299
Ready 8/299
Ready 9/299
Big value: 25 1552
Ready 10/299
Ready 11/299
Ready 12/299
Ready 13/299
Ready 14/299
Ready 15/299
Ready 16/299
Big value: 25 537
Ready 17/299
Ready 18/299
Ready 19/299
Ready 20/299
Ready 21/299
Ready 22/299
Big value: 25 6176
Ready 23/299
Ready 24/299
Ready 25/299
Ready 26/299
Ready 27/299
Big value: 20 550
Ready 28/299
Ready 29/299
Ready 30/299
Big value: 25 2601
Ready 31/299
Ready 32/299
Ready 33/299
Big value: 20 558
Ready 34/299
Ready 35/299
Ready 36/299
Ready 37/299
Big value: 25 3123
Ready 38/299
Ready 3

'for rest in one_rest[0:5]:\n    sli =  new_test[new_test["DaysOfRest"]== rest]\n    for name in names:\n        print name\n        val_dict = dict(sli[name].value_counts())\n        print val_dict'

In [7]:
# replace low freq city
big_city = ['İstanbul', 'Ankara', 'İzmir']
for i in range(0,len(train_normal)):
    if train_normal.iloc[i]["City"] not in big_city:
        train_normal.loc[i,"City"] = 'Other'

for i in range(0,len(test_normal)):
    if test_normal.iloc[i]["City"] not in big_city:
        test_normal.loc[i,"City"] = 'Other'    

In [59]:
# construct more datasets
# drop outliers
#train_normal = train_normal.drop("max_params",1)
outlier_index = [16,23, 74, 98, 99, 115, 132 ]

#23, 74, 98, 99, 115, 132    

train_clean = train_normal.copy()
train_clean = train_clean.drop('max_params',1)
#train_clean = train_clean.drop(train_clean.index[outlier_index])
#train_clean = train_clean.reset_index(drop=True)
print len(train_clean)

# all_categorial
all_categorial_clean = pd.get_dummies(train_clean)
all_categorial_clean["DaysOfRest"] = train_clean["DaysOfRest"]
all_categorial_clean = all_categorial_clean.drop('revenue', 1)
# all_simple
simple_clean = train_clean.copy()
simple_clean = pd.get_dummies(train_clean, columns = ["City","Type"])
simple_clean = simple_clean.drop('revenue', 1)

# no date
simple_clean_no_date = simple_clean.drop('DaysOfRest', 1)
all_categorial_clean_no_date = all_categorial_clean.drop('DaysOfRest', 1)

# null_not_null_values

null_simple = simple_clean[simple_clean["P14"] == 0]
not_null_simple = simple_clean[simple_clean["P14"] != 0]
null_simple = null_simple.reset_index(drop=True)
not_null_simple = not_null_simple.reset_index(drop=True)

null_categorical = train_clean[train_clean["P14"] == 0]
not_null_categorical = train_clean[train_clean["P14"] != 0]

null_categorical = pd.get_dummies(null_categorical)
null_categorical = null_categorical.drop('revenue', 1)
not_null_categorical = pd.get_dummies(not_null_categorical)
not_null_categorical = not_null_categorical.drop('revenue', 1)


null_categorical = null_categorical.reset_index(drop=True)
not_null_categorical =not_null_categorical.reset_index(drop=True)

null_revenue = train_clean[train_clean["P14"] == 0].revenue 
null_revenue = null_revenue.reset_index(drop=True)
not_null_revenue = train_clean[train_clean["P14"] != 0].revenue
not_null_revenue = not_null_revenue.reset_index(drop=True)

# normalized_data
#================================================================

# all_categorial
all_categorial_normal = pd.get_dummies(train_normal)
all_categorial_normal["DaysOfRest"] = train_normal["DaysOfRest"]
all_categorial_normal = all_categorial_normal.drop('revenue', 1)

# all_simple
simple_normal = train_normal.copy()
simple_normal = pd.get_dummies(train_normal, columns = ["City","Type"])
simple_normal = simple_normal.drop('revenue', 1)

# null_not_null
train_dummies = pd.get_dummies(train_clean)
dummy_revenue = train_dummies["revenue"]
null_categorical_normal = train_dummies[train_dummies["P14"] == 0].copy()
not_null_categorical_normal = train_dummies[train_dummies["P14"] != 0].copy()

null_categorical_normal = null_categorical_normal.drop('revenue', 1)
not_null_categorical_normal = not_null_categorical_normal.drop('revenue', 1)
train_dummies = train_dummies.drop('revenue', 1)

null_categorical_normal = null_categorical_normal.reset_index(drop=True)
not_null_categorical_normal = not_null_categorical_normal.reset_index(drop=True)

# make all not null forced
null_names = [name for name in null_categorical.keys() if null_categorical.iloc[0][name] == 0]
forced_null = train_dummies.copy()
for name in null_names:
    forced_null[name] = 0

130


In [19]:
print train_dummies.keys()

Index([u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest', u'City_Ankara', u'City_Other', u'City_İstanbul', u'City_İzmir', u'Type_DT', u'Type_FC', u'Type_IL'], dtype='object')


In [60]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
train_third = pd.DataFrame(poly.fit_transform(train_dummies))

In [4]:
# imputed values
train_imputed = pd.read_csv("raw/train_imputed_final.csv") # 137*43 matrix
test_imputed = pd.read_csv("raw/test_imputed_final.csv") # 100000*42 matrix

In [253]:
print train_imputed.shape, test_imputed.shape
print train_imputed.keys(), test_imputed.keys()
outlier_index = [16]

#23, 74, 98, 99, 115, 132    

train_clean = train_imputed.copy()
train_clean = train_clean.drop(train_clean.index[outlier_index])
train_clean = train_clean.reset_index(drop=True)
dummy_revenue = train_clean["revenue"]

train_clean = train_clean.drop('max_params',1)
train_clean = train_clean.drop('revenue',1)

#for name in train_clean.keys():
#    train_clean[name] = train_clean[name].astype('category')

train_clean = pd.get_dummies(train_clean)
test_clean = pd.get_dummies(test_imputed)

# add mean
train_clean["mean"] = pd.Series(np.repeat(np.log(np.mean(dummy_revenue)),len(train_clean)))
test_clean["mean"] = pd.Series(np.repeat(np.log(np.mean(dummy_revenue)),len(test_clean)))
# add flag for big shit
train_clean["big"] = pd.Series(np.repeat(0,len(train_clean)))
big_index = dummy_revenue[dummy_revenue>6000000].index
train_clean.loc[big_index,"big"] = pd.Series(np.repeat(1,len(big_index)), index = big_index)

# add flag to error shit
# error shit [1, 20, 55, 98, 115, 23, 74, 16, 132, 99, 78, 84, 63, 48, 46, 95, 117, 47, 71, 9, 96, 126]
error_list = [1, 20, 55, 98, 115, 23, 74, 16, 132, 99, 78, 84, 63, 48, 46, 95, 117, 47, 71, 9, 96, 126]

train_clean["errors"] = pd.Series(np.repeat(0,len(train_clean)))
err_index = pd.Index(error_list)
train_clean.loc[err_index,"errors"] = pd.Series(np.repeat(1,len(err_index)), index = err_index)

print train_clean.keys()
print test_clean.keys()
print set(train_clean.keys()) - set(test_clean.keys())
#train_clean = train_clean.drop(['P1',"P2","P3","P4","P5","P7","P9","P10","P11","P12","P13",
#                                "P14","P15","P16","P17","P18","P19","P21","P22","P29","P30","P26","P37"],1)
print train_clean.keys()

(137, 42) (100000, 40)
Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest', u'max_params'], dtype='object') Index([u'City', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest'], dtype='object')
Index([u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', 

In [254]:
# all in one
from sklearn import cross_validation, linear_model,ensemble
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn import grid_search

residuals = pd.Series()

dataset = train_clean.copy()
Y = np.log(dummy_revenue)

kf = cross_validation.KFold(len(dataset), n_folds=10,shuffle=True)
print kf


params = {'n_estimators': 1000, 'max_depth': 9, 'min_samples_split': 2,
          'learning_rate': 0.001, 'loss': 'ls', 'min_samples_leaf':3}
gbm = ensemble.GradientBoostingRegressor(**params)
params2 = {'n_estimators': 1000, 'max_depth': 9, 'min_samples_split': 2,
          'learning_rate': 0.001, 'loss': 'lad', 'min_samples_leaf':2, 'subsample':0.4}

regr = linear_model.LinearRegression()
elastic = linear_model.ElasticNetCV(l1_ratio=1, eps=0.0001, n_alphas=100, fit_intercept=True, normalize=True, precompute='auto', max_iter=1000, tol=0.0001, cv=8, copy_X=True)
gbm2 = ensemble.GradientBoostingRegressor(**params2)
ada_tr = ensemble.AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),n_estimators=100, loss='square')
knn = KNeighborsRegressor(n_neighbors=2, p=2, weights='distance')
knn2 = KNeighborsRegressor(n_neighbors=3,p=2, weights='uniform')
extr = ExtraTreeRegressor(criterion='mse', max_depth=10, min_samples_split=3, min_samples_leaf=3, max_features='auto')
rfr = ensemble.RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_leaf = 3)#, max_features=None)#)#,  
svr = SVR(C=1, epsilon=0.1)

grid_params = {
    'n_estimators':[10,100,200,300,500,1000],
    'min_samples_leaf':[3],
    'max_depth':[2,4,6,10]
}

#'max_features':('sqrt','auto'), 

grid = grid_search.GridSearchCV(ensemble.RandomForestRegressor(),grid_params,cv=10)

all_regr = [
    (gbm, 'GBM'),
    #(gbm2, 'GBM2'),
    (regr, 'Simple_regression'),
    (elastic, 'ElasticNet'),
    (knn, 'KNN'),
    #(knn2,'KNN2'),
    (ada_tr, 'AdaBoost'),
    #(grid, 'GridSearch'),
    (extr, 'ExtraTrees'),
    (rfr,'RandomForest'),
     (svr, 'SVR')
]
all_regr_ens = [
    #(gbm, 'GBM'),
    #(gbm2, 'GBM2'),
    (regr, 'Simple_regression'),
    (elastic, 'ElasticNet'),
    (knn, 'KNN'),
    #(knn2,'KNN2'),
    #(ada_tr, 'AdaBoost'),
    (extr, 'ExtraTrees'),
    (rfr,'RandomForest'),
    #(svr, 'SVR')
]

# grid search
#grid.fit(dataset, Y)
#print grid.best_estimator_
#print grid.best_params_


error_indexes = []
for multi_regr, regr_name in all_regr:
    print "============"
    print regr_name
    print "============"
    print multi_regr
    avg_rmse = []
    res = np.exp(Y)
    for train_index, test_index in kf:
        multi_regr.fit(dataset.iloc[train_index], Y[train_index])
        pred = np.exp(multi_regr.predict(dataset.iloc[test_index])) 
        rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
        avg_rmse.append(rmse)
        res.loc[test_index] = pred - np.exp(Y[test_index])
        print("RMSE: %.2f Max pred: %.2f" % (rmse,np.max(pred)))
    
    res = np.abs(res)
    res.sort(ascending=False)
    print res.index[0:10]
    error_indexes.extend(list(res[res>2000000].index))
    print "Total errors ",len(error_indexes)
    print "Big errors ", len(res[res>2000000])
    print np.max(res[0:10]), np.min(res[0:10])
    #multi_regr.fit(dataset,Y)
    
    print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))
    
"""print "Build ensemble"

ens_pred = ensemble.RandomForestRegressor(n_estimators=600, min_samples_leaf = 3, oob_score=True)
#ensemble.GradientBoostingRegressor(**params2)

error_indexes = []
avg_rmse = []
res = np.exp(Y)
for train_index, test_index in kf:
    dataset_extended = pd.DataFrame()
    for multi_regr, regr_name in all_regr_ens:
        #print "============"
        #print regr_name
        #print "============"
        #print multi_regr
        multi_regr.fit(dataset.iloc[train_index], Y[train_index])
        pred = np.exp(multi_regr.predict(dataset.iloc[test_index])) 
        rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
        #print("RMSE: %.2f Max pred: %.2f" % (rmse,np.max(pred)))     
        dataset_extended[regr_name] = multi_regr.predict(dataset)
    
    ens_pred.fit(dataset_extended.iloc[train_index], Y[train_index])
    pred = np.exp(ens_pred.predict(dataset_extended.iloc[test_index])) 
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f Max pred: %.2f" % (rmse,np.max(pred)))
    res.loc[test_index] = pred - np.exp(Y[test_index])
    
res = np.abs(res)
res.sort(ascending=False)
print res.index[0:10]
error_indexes.extend(list(res[res>2000000].index))
print "Total errors ",len(error_indexes)
print "Big errors ", len(res[res>2000000])
print np.max(res[0:10]), np.min(res[0:10])
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))
#res.hist()"""

sklearn.cross_validation.KFold(n=136, n_folds=10, shuffle=True, random_state=None)
GBM
GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.001,
             loss='ls', max_depth=9, max_features=None,
             max_leaf_nodes=None, min_samples_leaf=3, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
RMSE: 3002928.46 Max pred: 6618017.92
RMSE: 2258256.66 Max pred: 6260137.01
RMSE: 1463265.39 Max pred: 6212384.05
RMSE: 1236109.39 Max pred: 6759735.82
RMSE: 1116863.68 Max pred: 7074716.12
RMSE: 1381542.23 Max pred: 5885646.94
RMSE: 1159159.53 Max pred: 5397383.62
RMSE: 1543502.96 Max pred: 6541428.21
RMSE: 1058200.75 Max pred: 4113404.12
RMSE: 803776.66 Max pred: 8235772.01
Int64Index([74, 98, 99, 132, 48, 115, 23, 16, 131, 94], dtype='int64')
Total errors  16
Big errors  16
10086136.8421 2408184.4955
AVG RMSE: 1502360.57 MAX RMSE: 3002928.46 MIN RMSE: 803776.66
Simp

'print "Build ensemble"\n\nens_pred = ensemble.RandomForestRegressor(n_estimators=600, min_samples_leaf = 3, oob_score=True)\n#ensemble.GradientBoostingRegressor(**params2)\n\nerror_indexes = []\navg_rmse = []\nres = np.exp(Y)\nfor train_index, test_index in kf:\n    dataset_extended = pd.DataFrame()\n    for multi_regr, regr_name in all_regr_ens:\n        #print "============"\n        #print regr_name\n        #print "============"\n        #print multi_regr\n        multi_regr.fit(dataset.iloc[train_index], Y[train_index])\n        pred = np.exp(multi_regr.predict(dataset.iloc[test_index])) \n        rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))\n        #print("RMSE: %.2f Max pred: %.2f" % (rmse,np.max(pred)))     \n        dataset_extended[regr_name] = multi_regr.predict(dataset)\n    \n    ens_pred.fit(dataset_extended.iloc[train_index], Y[train_index])\n    pred = np.exp(ens_pred.predict(dataset_extended.iloc[test_index])) \n    rmse = np.sqrt(np.mean((pred - np.e

In [249]:
#print dummy_revenue[dummy_revenue>6000000].count()
#nice = dummy_revenue[dummy_revenue>6000000]
#dummy_revenue[dummy_revenue>6000000] = pd.Series(np.repeat(0,len(nice)))
#print dummy_revenue[dummy_revenue>6000000]
#print pd.Series(reduce(lambda x,y: x+y,error_indexes)).value_counts()
errors = pd.Series(error_indexes)
dict_errors = dict(errors.value_counts())
print type(dict_errors)
print len([key for key in errors.value_counts().keys() if errors.value_counts()[key] > 4])
print [key for key in errors.value_counts().keys() if errors.value_counts()[key] > 4]


<type 'dict'>
9
[74, 132, 98, 115, 61, 52, 9, 117, 39]


In [100]:
#  predict errors and outliers
# fill type with mean!!!
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
import random

normal_features = ['errors', 'big']
good_features = test_clean.keys()

all_samples = train_clean.copy()
all_samples = pd.concat([all_samples,test_clean])
good_train = all_samples.loc[:,good_features]
print good_train.shape

train_normal_imputed = train_clean.copy()
test_normal_imputed = test_clean.copy()
test_normal_imputed["errors"] = pd.Series(np.repeat(np.nan,len(test_normal_imputed)))
test_normal_imputed["big"] = pd.Series(np.repeat(np.nan,len(test_normal_imputed)))

for name in normal_features:
    print name
    print "***********************"
    # make train set
    imp_value = all_samples[name]
    train_imp = imp_value[pd.notnull(imp_value)]
    
    # make baselines
    vals = train_imp.value_counts()
    top_result = pd.Series(np.repeat(vals.keys()[0],len(train_imp)))
    random_result = pd.Series([vals.keys()[random.randint(0,len(vals.keys())-1)] for i in range(0,len(train_imp))])
    top_result = top_result.reset_index(drop=True)
    random_result = random_result.reset_index(drop=True)
    
    # get train things 
    X = good_train.iloc[train_imp.index]
    Y = train_imp
    X = X.reset_index(drop=True)
    Y = Y.reset_index(drop=True)
    
    # add mean expectation
    X[str(name+"_mean")] = top_result.reset_index(drop=True)
    Y = np.array(Y).astype(int) # fix FFSH problems
    
    # Create classifiers
    params = {'n_estimators':100, 'max_features':None, 'max_depth':6}
    rf = RandomForestClassifier(**params)
    ada = AdaBoostClassifier(n_estimators=100)
    knn = KNeighborsClassifier(n_neighbors=10, weights='distance')
    knn2 = KNeighborsClassifier(n_neighbors=100, weights='uniform')
    sgd = SGDClassifier(loss = 'log', alpha=0.01, n_iter=200)
    all_class = [(rf, 'RF'),
                (ada, 'Ada'),
                (knn, 'KNN'),
                (knn2,'KNN2'),
                (sgd, 'SGD')]

    # initial models 
    X_extended = X.copy()
    
    for multi_class, cl_name in all_class:
        print "Train: ", cl_name
        multi_class.fit(X, Y)
        X_extended[cl_name] = multi_class.predict(X)
        print "Extended with ",cl_name," shape ",X_extended.shape
    
    # consolidated model  
    print "Train consolidated model"
    cons_class =  OneVsRestClassifier(SGDClassifier(loss = 'log', alpha=0.01, n_iter=200))       
    cons_class.fit(X_extended, Y)
    
    # fill imputed values
    for df_to_fill in (train_normal_imputed,test_normal_imputed):
        accum_df = df_to_fill.copy()
        if 'revenue' in accum_df.keys():
            accum_df = accum_df.drop(['revenue','max_params'],1)
        goal_col = accum_df[name]
        goal_indexes = goal_col[pd.isnull(goal_col)].index
        
        # if nothing to impute
        if len(goal_indexes) == 0:
            print "Nothing to impute. Skip this imputation"
            continue
        
        accum_df = accum_df.loc[goal_indexes,good_features]
        accum_df = pd.get_dummies(accum_df)
        # add mean
        accum_df[str(name+"_mean")] = pd.Series(np.repeat(vals.keys()[0],len(accum_df)), index=goal_indexes)
        
        # fill missed keys
        for null_name in set(X.keys())-set(accum_df.keys()):
            accum_df[null_name] = pd.Series(np.repeat(0,len(accum_df)), index=goal_indexes)
        
        print "To predict shape: ",accum_df.shape, " Train shape ", X.shape
        
        base_accum_df = accum_df.copy()
        for multi_class, cl_name in all_class:
            print "Predict: ", cl_name
            accum_df[cl_name] = multi_class.predict(base_accum_df)
            print dict(accum_df[cl_name].value_counts(dropna=False))
            print "Extended with ",cl_name," shape ",accum_df.shape
        
        print "Predict consolidated"
        
        # predict consolidated
        df_to_fill.loc[goal_indexes, name] = cons_class.predict(accum_df)
        print dict(pd.Series(cons_class.predict(accum_df)).value_counts())
        
        print "Check for imputation"
        print dict(df_to_fill[name].value_counts(dropna=False))



(100136, 45)
errors
***********************
Train:  RF
Extended with  RF  shape  (136, 47)
Train:  Ada
Extended with  Ada  shape  (136, 48)
Train:  KNN
Extended with  KNN  shape  (136, 49)
Train:  KNN2
Extended with  KNN2  shape  (136, 50)
Train:  SGD
Extended with  SGD  shape  (136, 51)
Train consolidated model
Nothing to impute. Skip this imputation
To predict shape:  (100000, 46)  Train shape  (136, 46)
Predict:  RF
{0: 92666, 1: 7334}
Extended with  RF  shape  (100000, 47)
Predict:  Ada
{0: 83316, 1: 16684}
Extended with  Ada  shape  (100000, 48)
Predict:  KNN
{0: 99439, 1: 561}
Extended with  KNN  shape  (100000, 49)
Predict:  KNN2
{0: 100000}
Extended with  KNN2  shape  (100000, 50)
Predict:  SGD
{0: 86831, 1: 13169}
Extended with  SGD  shape  (100000, 51)
Predict consolidated
{0: 93276, 1: 6724}
Check for imputation
{0: 93276, 1: 6724}
big
***********************
Train:  RF
Extended with  RF  shape  (136, 47)
Train:  Ada
Extended with  Ada  shape  (136, 48)
Train:  KNN
Extended 

In [128]:
for name in train_normal_imputed.keys():
    print name
    print dict(train_normal_imputed[name].value_counts(dropna=False))
    print dict(test_normal_imputed[name].value_counts(dropna=False))

P1
{nan: 0, 1.0: 10, 2.0: 42, 3.0: 33, 4.0: 47, 5.0: 4}
{1: 5197, 2: 26562, 3: 25335, 4: 35147, 5: 7759}
P2
{nan: 0, 1.0: 5, 2.0: 13, 3.0: 16, 4.0: 39, 5.0: 63}
{nan: 0, 1.0: 1503, 2.0: 8719, 3.0: 17052, 4.0: 23009, 5.0: 49717}
P3
{nan: 0, 2.0: 3, 3.0: 11, 4.0: 104, 5.0: 18}
{nan: 0, 2.0: 1906, 3.0: 9962, 4.0: 78298, 5.0: 9834}
P4
{nan: 0, 3.0: 22, 4.0: 88, 5.0: 26}
{nan: 0, 3.0: 12376, 4.0: 66761, 5.0: 20863}
P5
{nan: 0, 1.0: 60, 2.0: 64, 3.0: 7, 4.0: 5}
{nan: 0, 1.0: 33685, 2.0: 50799, 3.0: 5931, 4.0: 9585}
P6
{nan: 0, 1.0: 12, 2.0: 61, 3.0: 28, 4.0: 17, 5.0: 18}
{1: 9850, 2: 43925, 3: 20554, 4: 16873, 5: 8798}
P7
{3.5: 4, 1.0: 12, 5.0: 120, nan: 0}
{3.5: 3748, 1.0: 9109, 5.0: 87143, nan: 0}
P8
{nan: 0, 3.0: 7, 4.0: 59, 5.0: 70}
{nan: 0, 3.0: 7767, 4.0: 45953, 5.0: 46280}
P9
{nan: 0, 4.0: 46, 5.0: 90}
{nan: 0, 4.0: 41808, 5.0: 58192}
P10
{nan: 0, 4.0: 40, 5.0: 96}
{4: 25209, 5: 74791}
P11
{nan: 0, 1.0: 22, 2.0: 46, 3.0: 29, 4.0: 18, 5.0: 21}
{1: 17781, 2: 23079, 3: 24118, 4: 17188, 5

In [250]:
Y = np.log(dummy_revenue)
#dataset = train_dummies.drop('revenue', 1)
fig = plt.figure()
ax1 = fig.add_subplot(111)
dataset = train_clean
#dataset = train_normal_imputed
#rfr.fit(dataset,Y)
#Y = np.log(train_clean.revenue)
ax1.scatter(np.exp(extr.predict(dataset)), np.exp(Y), c='b', marker="s", label='first')
ax1.scatter(np.exp(rfr.predict(dataset)),np.exp(Y), c='r', marker="o", label='second')
#ax1.scatter(np.exp(elastic.predict(dataset)),np.exp(Y), c='y', marker="<", label='third')
ax1.scatter(np.exp(Y),np.exp(Y), c='g', marker="*", label='fourth')

In [259]:
# make submission
dataset = train_normal_imputed
extr.fit(dataset,Y)
pred = np.exp(extr.predict(test_normal_imputed))

In [264]:
pred = pd.Series(pred)
print pred[pred>600000]

0     4005606.431186
1     3150737.732285
2     4005606.431186
3     3666884.187724
4     3512018.784610
5     4403549.807717
6     3137970.427244
7     4892273.957245
8     6681666.468508
9     2306968.469774
10    2502116.144919
11    2298282.065914
12    4552207.321924
13    2298282.065914
14    2306968.469774
...
99985    2883007.657781
99986    4892273.957245
99987    8653589.137294
99988    3150737.732285
99989    3666884.187724
99990    4623424.848871
99991    3785750.170798
99992    2306968.469774
99993    3666884.187724
99994    3785750.170798
99995    6681666.468508
99996    2306968.469774
99997    4552207.321924
99998    4739106.691353
99999    3006381.879520
Length: 100000, dtype: float64


In [222]:

for i in range(0,len(rfr.feature_importances_)):
    
    print train_normal_imputed.keys()[i], rfr.feature_importances_[i]*1000

P1 3.31509176026
P2 1.31708695963
P3 0.0
P4 0.0
P5 2.98660209153
P6 8.71813783613
P7 0.0
P8 7.28500338484
P9 0.0
P10 1.74993996665
P11 3.36932481419
P12 0.12612466989
P13 3.16352090321
P14 1.76744868585
P15 3.21152946147
P16 3.39281000753
P17 7.33103778233
P18 0.0
P19 0.928106528106
P20 5.4452321944
P21 0.0
P22 5.99512438173
P23 12.3107076708
P24 7.60128211999
P25 42.9265321886
P26 7.22246498581
P27 15.8863121608
P28 33.6444894265
P29 0.630166897225
P30 0.0
P31 10.5830102282
P32 6.24371798314
P33 7.132139098
P34 7.13256807082
P35 7.51395299057
P36 4.21206011312
P37 2.13656653084
DaysOfRest 156.696746391
City_Ankara 9.95876540147
City_Other 0.0
City_İstanbul 4.1498440924
City_İzmir 0.0
Type_FC 0.0
Type_IL 0.214575951318
mean 0.0
big 599.674566705
errors 4.02740956609


In [266]:
import csv
name = "./raw/my_submission_7_extr_imputed.csv"
result = open(name, "w")
writer = csv.writer(result, delimiter=',')
writer.writerow(["Id","Prediction"])

for i in range(0,len(pred)):
    writer.writerow([i, pred[i]])
result.close()    
submis = pd.read_csv(name)
print submis.shape

(100000, 2)


In [39]:
Y = dummy_revenue
dataset = train_third

pred = np.exp(clf.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "Tree: ", rmse

pred = np.exp(regr.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "Regr: ", rmse

pred = np.exp(elastic.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "Elastic: ", rmse

pred = np.exp(ada_tr.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "AdaBoost: ", rmse

pred = np.exp(knn.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "KNN: ", rmse

pred = np.exp(extr.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "Extra Trees: ", rmse

pred = np.exp(rfr.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "Random Forest: ", rmse

pred = np.exp(svr.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "SVR: ", rmse

pred = np.exp(sgd.predict(dataset))
rmse = np.sqrt(np.mean((pred - Y) ** 2))
print "SGD: ", rmse

Tree:  1243870.48577
Regr:  3.78472565211e-08
Elastic:  1666974.99961
AdaBoost:  754021.842087
KNN:  3.15176201539e-09
Extra Trees:  896100.081204
Random Forest:  1181328.89076
SVR:  688749.302114
SGD:  inf


In [48]:
pred = np.exp(extr.predict(dataset))
#for i in range(0, len(pred)):
#    print pred[i],Y[i], pred[i]-Y[i]
#diff = pred - Y
#diff.hist(bins=100)
models = [clf, regr, elastic, ada_tr, knn, extr, rfr, svr]
fig, axes = plt.subplots(nrows=3, ncols=3)
m = axes.flat
i = 0
for model in models:
    pred = np.exp(model.predict(dataset))
    diff = pred - Y
    m[i].plot(list(pred))#, bins = 100)
    i+=1
    
plt.show()

In [58]:
print dummy_revenue.mean()
print dummy_revenue.std()
outliers_index = dummy_revenue[dummy_revenue > dummy_revenue.mean()+2*dummy_revenue.std()].index
print train_dummies.iloc[outliers_index]
out_slice = train_dummies.iloc[outliers_index]
for name in train_dummies.keys():
    print name
    print dict(out_slice[name].value_counts())

4341448.74265
2225185.3415
     P1  P2  P3  P4  P5  P6  P7  P8  P9  P10   ...     P36  P37  DaysOfRest  \
23    5   5   3   5   2   2   5   5   5    4   ...       5    2           5   
74    5   5   4   5   1   4   5   3   4    4   ...       0    0           9   
98    4   5   4   4   2   4   5   4   4    4   ...       5    3          15   
99    3   5   4   4   2   5   5   5   5    5   ...       0    0           7   
115   2   4   4   4   2   2   5   4   4    5   ...       0    0           3   
132   4   5   4   4   2   3   5   4   4    5   ...       0    0           8   

     City_Ankara  City_Other  City_İstanbul  City_İzmir  Type_DT  Type_FC  \
23             0           0              1           0        0        0   
74             0           0              1           0        0        1   
98             0           0              1           0        0        0   
99             0           0              1           0        0        1   
115            0           0      

In [27]:
pred = clf.predict(dataset)
res = np.abs(np.exp(pred)-np.exp(Y))
res = pd.Series(res)
res.sort(ascending=False)
print res.index[0:20]
print res[0:10]
print res[len(res)-10:len(res)]
print res.mean()
print np.log(res.sum())
acc = set(res.index[0:20])

pred = elastic.predict(dataset)
res = np.abs(np.exp(pred)-np.exp(Y))
res = pd.Series(res)
res.sort(ascending=False)
print res.index[0:20]
print res[0:10]
print res[len(res)-10:len(res)]
print res.mean()
print np.log(res.sum())
acc = acc&set(res.index[0:20])

pred = regr.predict(dataset)
res = np.abs(np.exp(pred)-np.exp(Y))
res = pd.Series(res)
res.sort(ascending=False)
print res.index[0:20]
print res[0:10]
print res[len(res)-10:len(res)]
print res.mean()
print np.log(res.sum())
acc = acc&set(res.index[0:20])
#print train_clean.iloc[res.idxmax()]
#res.hist(bins=50)
print acc

train_clean.iloc[list(acc)]

ValueError: operands could not be broadcast together with shapes (137,) (49,) 

In [28]:
for name in names:
    print name
    print train_normal[name].value_counts().keys()

multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,'P11':2,'P12':2,'P13':1.5,'P14':3,
              'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5,
              'P28':2.5,'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}

# 3, 4, 7,9,10,12,13,18,29,34,35,36

P1
Float64Index([4.0, 2.0, 3.0, 1.0, 5.0], dtype='float64')
P2
Float64Index([5.0, 4.0, 3.0, 2.0, 1.0], dtype='float64')
P3
Float64Index([4.0, 5.0, 3.0, 2.0, 0.0], dtype='float64')
P4
Float64Index([4.0, 5.0, 3.0], dtype='float64')
P5
Float64Index([2.0, 1.0, 3.0, 4.0, 5.0], dtype='float64')
P6
Float64Index([2.0, 3.0, 5.0, 4.0, 1.0], dtype='float64')
P7
Float64Index([5.0, 1.0, 4.0, 3.0], dtype='float64')
P8
Float64Index([5.0, 4.0, 3.0, 2.0, 1.0], dtype='float64')
P9
Float64Index([5.0, 4.0], dtype='float64')
P10
Float64Index([5.0, 4.0], dtype='float64')
P11
Float64Index([2.0, 3.0, 1.0, 5.0, 4.0], dtype='float64')
P12
Float64Index([5.0, 4.0, 3.0, 2.0], dtype='float64')
P13
Float64Index([5.0, 4.0, 3.0], dtype='float64')
P14
Float64Index([0.0, 3.0, 1.0, 2.0, 4.0, 5.0], dtype='float64')
P15
Float64Index([0.0, 4.0, 2.0, 5.0, 1.0, 3.0], dtype='float64')
P16
Float64Index([0.0, 4.0, 3.0, 5.0, 2.0, 1.0], dtype='float64')
P17
Float64Index([0.0, 1.0, 2.0, 5.0, 3.0, 4.0], dtype='float64')
P18
Float64I

In [29]:
missed_values = ['P4','P7','P9','P18','P29', 'P34', 'P36']
for name in missed_values:
    print name
    print train_normal[name].value_counts()

P4
4    89
5    26
3    22
dtype: int64
P7
5    121
1     12
4      3
3      1
dtype: int64
P9
5    91
4    46
dtype: int64
P18
0    88
4    35
3     9
1     3
5     2
dtype: int64
P29
3    94
2    24
1    17
0     2
dtype: int64
P34
0    88
3    26
4    14
2     6
5     3
dtype: int64
P36
0    88
3    28
4    12
5     8
2     1
dtype: int64


In [272]:
print all_categorial_normal.keys()
all_categorial_normal =all_categorial_normal.drop('max_params',1)

Index([u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest', u'max_params', u'City_Adana', u'City_Afyonkarahisar', u'City_Amasya', u'City_Ankara', u'City_Antalya', u'City_Aydın', u'City_Balıkesir', u'City_Bolu', u'City_Bursa', u'City_Denizli', u'City_Diyarbakır', u'City_Edirne', u'City_Elazığ', u'City_Eskişehir', u'City_Gaziantep', u'City_Isparta', u'City_Karabük', u'City_Kastamonu', u'City_Kayseri', u'City_Kocaeli', u'City_Konya', u'City_Kütahya', u'City_Kırklareli', u'City_Muğla', u'City_Osmaniye', u'City_Sakarya', u'City_Samsun', u'City_Tekirdağ', u'City_Tokat', u'City_Trabzon', u'City_Uşak', u'City_İstanbul', u'City_İzmir', u'City_Şanlıurfa', u'City Group_Big Cities', u'City Group_Other', u'Type_DT', u'Type_FC', u'Type_IL'], dtype='object